In [18]:
import pandas as pd
import numpy as np

class NBAPredictor:
    def __init__(self):
        self.current_stats = pd.read_csv('new_szn.txt')
        self.indexed_short = self.current_stats[['Tm', 'Player', 'MP', 'PTS', 'AST', 'DRB', 'ORB', 'TRB', 'TOV', 'FG%', 'STL']].set_index('Tm')
        self.player_indexed = self.current_stats[['Tm', 'Player', 'MP', 'PTS', 'AST', 'DRB', 'ORB', 'TRB', 'TOV', 'FG%', 'STL']].set_index('Player')
        self.no_inactives = pd.DataFrame(data=None)
        self.inactive_dict = dict()
        self.team1boards = float
        self.team1assists = float
        self.team1points = float
        self.team2boards = float
        self.team2assists = float
        self.team2points = float
        self.team1score = 101.3
        self.team2score = 98.7
        self.team_abb_dict = {'Milwaukee Bucks':'MIL',
                              'Los Angeles Lakers': 'LAL',
                             'Boston Celtics': 'BOS',
                             'Los Angeles Clippers':'LAC',
                             'Utah Jazz':'UTA',
                             'Phoenix Suns': 'PHO',
                             'Miami Heat': 'MIA',
                             'Toronto Raptors': 'TOR',
                             'Philadelphia 76ers': 'PHI',
                             'Denver Nuggets': 'DEN',
                             'Dallas Mavericks': 'DAL',
                             'Oklahoma City Thunder': 'OKC',
                             'Houston Rockets': 'HOU',
                             'Indiana Pacers': 'IND',
                             'Portland Trail Blazers': 'POR',
                             'Cleveland Cavaliers': 'CLE',
                             'Orlando Magic': 'ORL',
                             'Minnesota Timberwolves': 'MIN',
                             'San Antonio Spurs': 'SAS',
                             'Sacramento Kings': 'SAC',
                             'Atlanta Hawks': 'ATL',
                             'Brooklyn Nets': 'BRK',
                             'Washington Wizards': 'WAS',
                             'New Orleans Pelicans': 'NOP',
                             'Detroit Pistons': 'DET',
                             'Charlotte Hornets': 'CHO',
                             'Chicago Bulls': 'CHI',
                             'Golden State Warriors': 'GSW',
                             'Memphis Grizzlies': 'MEM',
                             'New York Knicks': 'NYK'}
        self.backwards_team_dict = {'Lakers Los Angeles': 'LAL',
                                    'Bucks Milwaukee':'MIL',
                             'Celtics Boston': 'BOS',
                             'Clippers Los Angeles':'LAC',
                             'Jazz Utah':'UTA',
                             'Suns Phoenix': 'PHO',
                             'Heat Miami': 'MIA',
                             'Raptors Toronto': 'TOR',
                             '76ers Philadelphia': 'PHI',
                             'Nuggets Denver': 'DEN',
                             'Mavericks Dallas': 'DAL',
                             'Thunder Oklahoma City': 'OKC',
                             'Rockets Houston': 'HOU',
                             'Pacers Indiana': 'IND',
                             'Trail Blazers Portland': 'POR',
                             'Cavaliers Cleveland': 'CLE',
                             'Magic Orlando': 'ORL',
                             'Timberwolves Minnesota': 'MIN',
                             'Spurs San Antonio': 'SAS',
                             'Kings Sacramento': 'SAC',
                             'Hawks Atlanta': 'ATL',
                             'Nets Brooklyn': 'BRK',
                             'Wizards Washington': 'WAS',
                             'Pelicans New Orleans': 'NOP',
                             'Pistons Detroit': 'DET',
                             'Hornets Charlotte': 'CHO',
                             'Bulls Chicago': 'CHI',
                             'Warriors Golden State': 'GSW',
                             'Grizzlies Memphis': 'MEM',
                             'Knicks New York': 'NYK'}
        

    def input_teams(self):
        self.team1 = input('Enter home team abbreviation')
        self.team2 = input('Enter away team abbreviation')
        
    def team_stat_effects(self):
        teams = pd.read_csv('team_stats.txt')
        #USES THE DICTIONARY TO MAKE THE ABBREVIATION INTO A FULL NAME THAT CAN BE USED
        for full in self.team_abb_dict.keys():
            if self.team_abb_dict[full] == self.team1:
                fullteam1 = full
        for full in self.team_abb_dict.keys():
            if self.team_abb_dict[full] == self.team2:
                fullteam2 = full
        team_indexed = teams.set_index('Team')
        o_rating1 = team_indexed.loc[fullteam1]['ORtg/A']
        o_rating2 = team_indexed.loc[fullteam2]['ORtg/A']
        d_rating1 = team_indexed.loc[fullteam1]['DRtg/A']
        d_rating2 = team_indexed.loc[fullteam2]['DRtg/A']
        #GETS OFFENSIVE RATING AND DEFENSIVE RATING AND TAKES THE DIFFERENCE OF THE TEAMS, SUBTRACTS IT FROM 8 AND
        o_adv1 = (8 - (o_rating1-d_rating2))
        o_adv2 = (8 - (o_rating2 - d_rating1))
        self.team1score += o_adv1
        self.team2score += o_adv2
        
    def adv_team_stat_effects(self):
        adv_team = pd.read_csv('adv_team_stats.txt').set_index('Team')
        for full in self.backwards_team_dict.keys():
            if self.backwards_team_dict[full] == self.team1:
                team1 = full
        for full in self.backwards_team_dict.keys():
            if self.backwards_team_dict[full] == self.team2:
                team2 = full
        pace1 = adv_team.loc[team1]['PACE']
        pace2 = adv_team.loc[team2]['PACE']
        self.team1score = self.team1score + (pace1-pace2)*.2
        self.team2score = self.team2score + (pace2-pace1)*.2
        team1oreb = adv_team.loc[team1]['OREB%']
        team2oreb = adv_team.loc[team2]['OREB%']
        self.team1score += (team1oreb*.2)
        self.team2score += (team2oreb*.2)
        efg1 = adv_team.loc[team1]['EFG%']
        efg2 = adv_team.loc[team2]['EFG%']
        self.team1score = self.team1score + ((efg1 - .510)*150)
        self.team2score = self.team2score + ((efg2 - .510)*150)
        oppefg1 = adv_team.loc[team1]['EFG%ALWD']
        oppefg2 = adv_team.loc[team2]['EFG%ALWD']
        self.team2score = self.team2score + ((oppefg1 - .520)*125)
        self.team1score = self.team1score + ((oppefg2 - .520)*125)
        team1TOadv = (adv_team.loc[team1]['TO%ALWD'] + adv_team.loc[team2]['TO%'])*.2
        team2TOadv = (adv_team.loc[team2]['TO%ALWD'] + adv_team.loc[team1]['TO%'])*.2
        self.team1score += (team1TOadv - team2TOadv)*.5
        self.team2score += (team2TOadv - team1TOadv)*.5
        
    def input_inactives(self):
        x = 0
        y = 0
        in_between = self.current_stats.set_index('Player')
        while x == 0:
            inactive = input('Inactive players')
            if inactive == 'none':
                x += 1
            elif in_between.loc[inactive]['Tm'] == self.team1:
                self.team1score -= (in_between.loc[inactive]['PTS'] + in_between.loc[inactive]['TRB'] + in_between.loc[inactive]['AST'])*(in_between.loc[inactive]['MP']*.005)
            elif in_between.loc[inactive]['Tm'] == self.team2:
                self.team2score -= (in_between.loc[inactive]['PTS'] + in_between.loc[inactive]['TRB'] + in_between.loc[inactive]['AST'])*(in_between.loc[inactive]['MP']*.005)
        
    def top_points(self):
        team1list = self.indexed_short.loc[self.team1].sort_values(['PTS'], ascending=False).head(5)
        team1agg = team1list.groupby(['Tm']).sum()
        self.team1TOPpoints = team1agg.loc[self.team1]['PTS']
        team2list = self.indexed_short.loc[self.team2].sort_values(['PTS'], ascending=False).head(5)
        team2agg = team2list.groupby(['Tm']).sum()
        self.team2TOPpoints = team2agg.loc[self.team2]['PTS']
        
    def bench_points(self):
        team1list = self.indexed_short.loc[self.team1].sort_values(['PTS'], ascending=False)[5:9]
        team1agg = team1list.groupby(['Tm']).sum()
        self.team1BENCHpoints = team1agg.loc[self.team1]['PTS']
        team2list = self.indexed_short.loc[self.team2].sort_values(['PTS'], ascending=False)[5:9]
        team2agg = team2list.groupby(['Tm']).sum()
        self.team2BENCHpoints = team2agg.loc[self.team2]['PTS']
        
    def total_rebounds(self):
        team1list = self.indexed_short.loc[self.team1].sort_values(['MP'], ascending=False).head(9)
        team1agg = team1list.groupby(['Tm']).sum()
        self.team1boards = team1agg.loc[self.team1]['TRB']
        team2list = self.indexed_short.loc[self.team2].sort_values(['MP'], ascending=False).head(9)
        team2agg = team2list.groupby(['Tm']).sum()
        self.team2boards = team2agg.loc[self.team2]['TRB']
        
    def total_assists(self):
        team1list = self.indexed_short.loc[self.team1].sort_values(['MP'], ascending=False).head(9)
        team1agg = team1list.groupby(['Tm']).sum()
        self.team1assists = team1agg.loc[self.team1]['AST']
        team2list = self.indexed_short.loc[self.team2].sort_values(['MP'], ascending=False).head(9)
        team2agg = team2list.groupby(['Tm']).sum()
        self.team2assists = team2agg.loc[self.team2]['AST']
        
    def predict_score(self):
        return print(self.team1 + ': ' + str(round(self.team1score)) + '\n' + self.team2 + ': ' + str(round(self.team2score)))
    
    def winning_percentage(self):
        point_dif = self.team1score - self.team2score
        team1winp = 50 + (point_dif/70)*100
        team2winp = 50 - (point_dif/70)*100
        return print('Home team win percentage: ' + str(team1winp) + '%' + '\n' + 'Away team win percentage: ' + str(team2winp) + '%')
    
    def main(self):
        self.input_teams()
        self.input_inactives()
        self.team_stat_effects()
        self.adv_team_stat_effects()
        self.top_points()
        self.bench_points()
        self.total_rebounds()
        self.total_assists()
        self.predict_score()
        self.winning_percentage()
        
        


In [19]:
yer = NBAPredictor()
yer.main()

Enter home team abbreviationDEN
Enter away team abbreviationHOU
Inactive players


KeyError: 'the label [] is not in the [index]'

In [15]:
lmno = pd.read_csv('adv_team_stats.txt')
lmno

,Team,NOTSURE,Ortg,Drtg,PACE,FTRATE,3fgtend,TS%,EFG%,TO%,OREB%,FT/FGA,EFG%ALWD,TO%ALWD,DREBALWD,FT/FGAALWD
0,Wizards Washington,11,113.2,115.2,104.3,20.5,36.7,0.568,0.535,12.4,24.1,16.8,0.545,13.5,75.9,22.8
1,Jazz Utah,13,104.1,100.7,100.3,32.2,35.9,0.556,0.513,14.7,18.0,24.5,0.481,13.2,82.0,20.0
2,Raptors Toronto,13,109.9,102.9,102.3,24.3,41.5,0.583,0.544,14.3,18.6,20.2,0.477,13.1,81.4,18.1
3,Spurs San Antonio,14,110.4,112.7,101.7,26.9,27.6,0.554,0.513,12.1,24.8,21.2,0.534,11.0,75.2,20.8
4,Kings Sacramento,13,106.7,110.7,98.0,22.8,40.7,0.558,0.525,14.8,24.4,17.7,0.543,14.0,75.6,21.5
5,Trail Blazers Portland,15,108.1,110.9,103.0,24.4,36.7,0.547,0.506,12.6,22.2,19.8,0.504,12.2,77.8,24.9
6,Suns Phoenix,13,109.6,104.5,103.5,28.2,40.2,0.583,0.542,13.6,18.2,22.7,0.520,15.5,81.8,24.0
7,76ers Philadelphia,13,107.1,104.1,101.7,24.6,34.1,0.559,0.530,15.2,23.7,18.0,0.517,14.2,76.3,24.6
8,Magic Orlando,13,104.7,104.0,97.6,25.3,35.6,0.523,0.480,11.7,23.3,20.2,0.505,12.6,76.7,15.9
9,Thunder Oklahoma City,14,105.2,105.3,99.4,30.6,35.4,0.562,0.517,14.4,18.8,24.0,0.503,13.7,81.3,18.0


In [16]:
class StatUpdates:
    
    def clean_team_stats(self):
        xlist = list()
        f = open('new_stats_in.txt','r', encoding='UTF-8')
        lszn = f.readlines()
        for player in lszn[1:]:
            splayer = player.split(',')
            xlist.append(splayer)
        for lst in xlist:
            lst.pop(0)
        f.close()
        xf = open('team_stats.txt', 'w', encoding='UTF-8')
        xf.write('Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A\n')
        for y in xlist:
            for x in y:
                if '\n' in x:
                    xf.write(x)
                else:
                    xf.write(x + ',')
        xf.close()
        
    def update_adv_stats(self):
        adv_team = open('adv_team_stats_in.txt', 'r', encoding='UTF-8')
        adv_team_stats = open('adv_team_stats.txt', 'w', encoding='UTF-8')
        adv_lines = adv_team.readlines()
        for line in adv_lines:
            line = line.split('\t')
            for stat in line:
                if '\n' in stat:
                    adv_team_stats.write(stat)
                else:
                    adv_team_stats.write(stat + ',')
        adv_team.close()
        adv_team_stats.close()
        
    def update_stats(self):
        nlist = list()
        f = open('current_stats.txt','r', encoding='UTF-8')
        lszn = f.readlines()
        for player in lszn[1:]:
            splayer = player.split(',')
            nlist.append(splayer)
        for lst in nlist:
            lst.pop(0)
        for x in nlist:
            div = x[0].find('\\')
            x[0] = x[0].replace(x[0][div:], '')
        newf = open('new_szn.txt', 'w', encoding='UTF-8')
        newf.write('Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS\n')
        for player in nlist[:-1]:
            for stat in player:
                if '\n' in stat:
                    newf.write(stat)
                else:
                    newf.write(stat + ',')
        newf.close()
        f.close()
        
    def main(self):
        self.update_stats()
        self.update_adv_stats()
        self.clean_team_stats()

        

In [17]:
ha = StatUpdates()
ha.main()

FileNotFoundError: [Errno 2] No such file or directory: 'current_stats.txt'